In [1]:
# Importing libraries

In [2]:
# Check env vars
!env | grep -e "SPARK" -e "PYTHON"

PYSPARK_DRIVER_PYTHON=/Users/c11309a/.local/share/rtx/installs/python/3.10/bin/python
PYSPARK_PYTHON=/Users/c11309a/.local/share/rtx/installs/python/3.10/bin/python
PYTHONPATH=/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/pyspark.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/*.zip:
SPARK_HOME=/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3
PYTHONUNBUFFERED=1
PYTHONIOENCODING=utf-8
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING=1


In [3]:
# Create a spark session
from pyspark.sql import SparkSession

spark = (
            SparkSession.builder.appName("learn_import")
                .master("local[4]")
                .getOrCreate()
        )

sc = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/27 23:58:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# Import the parser lbrary from learnspark module
from learnspark.parser import parse_line_factory
from learnspark.schemas import load_schema
from pyspark.sql.types import StructType
from pyspark.sql.functions import udf

# Load the schema
schema: StructType = load_schema('data/fwf_fields.yaml')

# Create udf to parse lines
parse_line_udf = udf(parse_line_factory(schema), schema)

In [8]:
# Read the file
df = spark.read.text('data/people.txt')
df.show(truncate=False)

+------------------+
|value             |
+------------------+
|1    John Doe  025|
|2    Jane Doe  030|
|3    Jim Smith 035|
|4    Ann Brown 040|
|5    Tom Davis 045|
+------------------+



In [9]:
# Parse the data
df = df.withColumn('parsed', parse_line_udf('value')).select('parsed.*')
df.show(truncate=False)

+---+---------+---+
|id |name     |age|
+---+---------+---+
|1  |John Doe |25 |
|2  |Jane Doe |30 |
|3  |Jim Smith|35 |
|4  |Ann Brown|40 |
|5  |Tom Davis|45 |
+---+---------+---+

